In [ ]:
import requests
import requests
import json
import csv
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

GET TOP 5 VIDEO WITH MOST COMMENTS

In [ ]:
root_path = "CSV_PATH"

top_per_party = {}


for party in os.listdir(root_path):
    party_path = os.path.join(root_path, party)
    if not os.path.isdir(party_path):
        continue

    party_dfs = []

    for file in os.listdir(party_path):
        if file.endswith('.csv'):
            file_path = os.path.join(party_path, file)
            try:
                df = pd.read_csv(file_path, delimiter="\t")
                df['party'] = party
                if 'comment_count' in df.columns and 'id' in df.columns:
                    party_dfs.append(df[['id', 'comment_count', 'party']])
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

    # Join each file (month) and order by comment_count
    if party_dfs:
        party_df = pd.concat(party_dfs, ignore_index=True)
        party_df = party_df.dropna(subset=['comment_count'])
        party_df['comment_count'] = party_df['comment_count'].astype(int)
        top_per_party[party] = party_df.sort_values(by='comment_count', ascending=False).head(5)



print("Top 5 by party:")
for party, df in top_per_party.items():
    print(f"\n{party}:")
    print(df)

GET COMMENTS DATA

In [ ]:
#request access_token
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Cache-Control': 'no-cache',
}

# Fill with TikTok API credentials. Can be requested at https://developers.tiktok.com/products/research-api/
data = {
    'client_key': 'API_CREDENTIALS',
    'client_secret': 'API_CREDENTIALS',
    'grant_type': 'client_credentials',
}

response = requests.post('https://open.tiktokapis.com/v2/oauth/token/', headers=headers, data=data).json()
auth = response['access_token']

In [ ]:
def get_comments(video_id, auth):

    headers = {
        "authorization": f"bearer {auth}",
        "Content-Type": "application/json",
    }

    data = {
        "video_id": str(video_id),
        "max_count": 50,
        "cursor": 0
    }

    response = requests.post(
        "https://open.tiktokapis.com/v2/research/video/comment/list/?fields=id,like_count,create_time,text,video_id,parent_comment_id",
        headers=headers, data=json.dumps(data)
    ).json()

    return response


In [ ]:
def write_comments_file(folder_path, filename, comments_data):
    if 'data' in comments_data and 'comments' in comments_data['data']:
        comments = comments_data['data']['comments']
    else:
        print("No comments found in response data.")
        return

    os.makedirs(folder_path, exist_ok=True)
    filepath = os.path.join(folder_path, filename)

    all_keys = set()
    for comment in comments:
        all_keys.update(comment.keys())

    fieldnames = sorted(all_keys)

    # Save in CSV
    with open(filepath, 'w', newline='', encoding='utf-8') as csvFile:
        csvwriter = csv.DictWriter(csvFile, fieldnames=fieldnames, delimiter="\t")
        csvwriter.writeheader()
        for comment in comments:
            csvwriter.writerow(comment)

    print(f"Comments saved to {filepath}")


In [ ]:
video_id = "VIDEO_ID"
comments_data = get_comments(video_id, auth)

folder_path = "PARTY_PATH"
filename = "FILENAME"

write_comments_file(folder_path, filename, comments_data)
